# 🚆 Hackathon: Prediksjon av Togforsinkelser
Velkommen til dette hackathon-caset! Målet er å bygge en modell som kan predikere forsinkelser i togtrafikken. Vi gir deg et datasett og forslag til steg for å komme i gang. 

## 1. 📊 EDA (Utforskende Dataanalyse)
I denne seksjonen skal du bli kjent med datasettet. Målet er å forstå hvilke variabler som finnes, hvordan dataene er fordelt, og om det finnes åpenbare feil eller mangler.

**Eksempelsteg:**
* Skriv ut de første radene i datasettet (df.head())

* Undersøk datatyper og kolonnenavn (df.info())

* Sjekk for manglende verdier og outliers

* Visualiser fordelingen av viktige variabler (f.eks. histogrammer, boksplott)

* Se på sammenhenger mellom variabler (f.eks. korrelasjonsmatrise)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Last inn datasettet
df = pd.read_csv('../data/data_train.csv', sep=";") # Endre filsti og delimiter ved behov

# Se på de første radene
df.head()

## 2. 🧹 Preprosesering
Før du kan trene en modell, må dataene være rene og konsistente. Her håndterer du manglende verdier, konverterer datatyper og fjerner irrelevante eller dupliserte kolonner. Her utføres nødvendige steg oppdaget i datainspiseringen i forrige steg.

**Eksempelsteg**

* Identifiser og håndter manglende verdier, enten ved å fylle inn med passende verdier (f.eks. median, modus) eller ved å fjerne rader/kolonner med for mange mangler.

* Fjerne outliers 

* Konverter kolonner til riktige datatyper (f.eks. dato/tid, kategoriske variabler (ved f.eks. one-hot encoding))

* Fjern eller transformer irrelevante kolonner.  Datasettet inneholder mange kolonner som kanskje ikke er så relevante for forsinkelsen. Behold kun de som tilfører modellen informasjon.

* Utfør eventuell feature engineering, for eksempel å lage nye variabler basert på eksisterende data (se under for utregnign av strekning)

* Utfør eventuell normalisering av verdier 


#### Domenekunnskap
**Utregning av togstrekning/retning (kan brukes som feature)**

Datasettet inneholder ingen informasjon om hvilken retning toget kjører i. Det finnes også forskjellige strekninger på L1 linjen. Dette kan regnes ut ved bruk av tognummer på måten beskrevet under basert på `TRAIN_ID`:

| Betingelse                                                                 | Retning                  |
|----------------------------------------------------------------------------|--------------------------|
| 2100–2199 og oddetall                                                      | Lillestrøm–Asker         |
| 2100–2199 og partall                                                       | Asker–Lillestrøm         |
| 2204                                                                       | Asker–Lillestrøm         |
| 2200–2299 og oddetall (unntatt 2281)                                       | Lillestrøm–Spikkestad    |
| 2200–2299 og partall (unntatt 2204)                                        | Spikkestad–Lillestrøm    |
| 2281                                                                     | Nattog              |

**Adgangstider**

Merk at start-stasjoner mangler tidsstempler for `PLANNED_ARRIVAL_TIME` og endestasjon mangler tidsstempel for `PLANNED_ARRIVAL_TIME`.

**`DELAY_SEC`**

En forsinkelse regnes kun som en forsinkelse dersom den er større eller lik 240 sekunder (4 minutter) og rundes alltid ned til hele minutt målt i sekunder. Derfor vil verdiene i datasettet alltid være multipler av 60, med nedre grense på 240. 

In [ ]:
# Sjekk for manglende verdier
df.isnull().sum()

# Fyll inn eller fjern manglende verdier
df = df.dropna()  # Eksempel: fjerner rader med manglende verdier

# Konverter kolonner hvis nødvendig


# df['tidspunkt'] = pd.to_datetime(df['tidspunkt'])

# ONE-HOT ENCODING
df = pd.get_dummies(df, columns=['RESPONSIBLE_CODE'], drop_first=True)

## 3. 🧠 Modelloppsett
I denne fasen velges hvilke variabler (features) som skal brukes for å predikere målet (target), og det settes opp en maskinlæringsmodell. Datasettet deles i trenings- og testsett for å kunne evaluere modellens generaliseringsevne. Husk at for Kaggle-konkurransen skal du kun bruke treningsdata til å trene modellen, og testdata til å generere prediksjoner for innsending.

**Eksempelsteg:**
* Velg relevante features og definer target-variabelen.

* Del datasettet i trenings- og testsett ved hjelp av for eksempel `train_test_split` for å sikre uavhengig evaluering.

* Velg og initialiser en passende modell (f.eks. Linear Regression, Random Forest, XGBoost).

* Bestem hyperparametere for valgt modell

* Tren modellen på treningsdataene ved å kalle på en `.fit()`- eller `.train()`-metode

Under har vi brukt Linear regression som eksempel.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Definer features og target
features = ['REGISTERED_EXTRA_DELAY_SEC/60', 'TRAIN_ID', 'DEVIATION_REASON_CODE']  # Eksempel
target = 'DELAY_SEC'

X = df[features]
y = df[target]

# Del opp i trenings- og testdata
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bestem hyperparametere for valgt modell


# Tren modellen
model = LinearRegression()
model.fit(X_train, y_train)

## 4. 🔮 Predikering

Etter at modellen er trent, brukes den til å lage prediksjoner på testdataen `X_test`. Dette gir et inntrykk av hvordan modellen presterer på ny, usett data. 


Bruk den trente modellen til å predikere målverdier for testsettet (`model.predict`).


In [ ]:
# Generer prediksjoner
y_pred = model.predict(X_test)

# Vis noen prediksjoner
y_pred[:10]

## 5. 📈 Evaluering
Evaluer modellens ytelse ved hjelp av metrikker. Konkurransen vurderes ved bruk av RMSE. Det kan være lurt å bruke samme evalueringsmetrikk for å kunne sammenligne resultatene direkte.

**Eksempelsteg:**

* Beregn og rapporter MAE og RMSE for testsettet.

* Diskuter resultatene og hva de betyr for modellens praktiske anvendelse.

* Visualiser prediksjoner mot faktiske verdier for å identifisere systematiske feil eller forbedringspotensial.

* Sammenlign flere modeller dersom det er aktuelt.



In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

# Visualiser
plt.figure(figsize=(10, 6))
plt.plot(y_test.reset_index(drop=True), label='True Values')
plt.plot(y_pred, label='Linear Regression', alpha=0.7)

plt.legend()
plt.title('Model Prediction vs True Values')
plt.xlabel('Sample Index')
plt.ylabel('Delay (seconds)')
plt.show()

## 5. ✅ Submission
Når dere har landet på en modell dere er fornøyd med lokalt, kan dere bruke denne til å predikere på dataen dere finner i data_test.csv.

⚠️ **Viktig:** Denne dataen må også gjennom cleaning og prepping, men pass på å ikke filtrere bort noen rader dersom det inngår som en del av rensingen da dette vil medføre at evalueringen av løsningen deres ikke vil fungere.

Prediksjonene skal lastes opp i kagglekonkurransen på formatet (ID, DELAY_SEC) hvor ID hentes fra testdatasettet. Se eksempelkode under:


In [ ]:
# Les inn testdatasett
df_test = pd.read_csv('../data/data_test.csv', sep=";") # Endre filsti og delimiter ved behov
# Utfør samme datapreparering som på treningsdataen for testdatasett

# Bruk samme features som tidligere
X_submission = df_test[features]

# Prediker for testdatasettet
pred_submission = model.predict(X_submission)

# Bruk ID fra testdatasett
submission = pd.DataFrame({
    "ID": df_test['ID'],
    "DELAY_SEC": pred_submission
})
# Lagre til CSV
submission.to_csv("../submissions/linreg_predictions_testdata.csv", index=False)
